In [1]:
import numpy as np
import pandas as pd
from scipy import stats

### Задача 1

#### Даны значения величины заработной платы заемщиков банка (salary) и значения их поведенческого кредитного скоринга (scoring):

salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]

scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]
#### Используя только встроенные питоновские функции и структуры данных (т.е. без библиотек numpy, pandas и др.) найдите:
    1. ковариацию (смещённую и несмещённую) этих двух величин,
    2. коэффициент корреляции Пирсона.
#### Можно затем посчитать те же значения с использованием библиотек, чтобы проверить себя.

In [2]:
salary = [35, 45, 190, 200, 40, 70, 54, 150, 120, 110]
scoring = [401, 574, 874, 919, 459, 739, 653, 902, 746, 832]

1. ковариацию (смещённую и несмещённую) этих двух величин

$\sigma_{XY} = \dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n (x_i - \overline{X}) \cdot (y_i - \overline{Y})$

In [3]:
def cov(X, Y, ddof=0):
    '''covariance'''
    
    index = range(len(X))
    MX = sum(X)/len(X)
    MY = sum(Y)/len(Y)
    
    Sum = sum([(X[i] - MX) * (Y[i] - MX) for i in index])
    
    return Sum/(len(X)-ddof)

In [4]:
cov(X=salary, Y=scoring, ddof=0) # biased

9157.84

In [5]:
np.cov(salary, scoring, ddof=0) # check

array([[ 3494.64,  9157.84],
       [ 9157.84, 30468.89]])

In [6]:
cov(X=salary, Y=scoring, ddof=1) # unbiased

10175.377777777778

In [7]:
np.cov(salary, scoring, ddof=1) # check

array([[ 3882.93333333, 10175.37777778],
       [10175.37777778, 33854.32222222]])

2. коэффициент корреляции Пирсона.

$r_{XY} = \dfrac{\sigma_{XY}}{\sigma_X \cdot \sigma_Y}$

$\sigma_X = \sqrt{\dfrac{1}{n} \displaystyle\sum_{i = 1}^n \left( x_i - \overline{X} \right)^2}, \:\:
\sigma_{X, \: unbiased} = \sqrt{\dfrac{1}{n - 1} \displaystyle\sum_{i = 1}^n \left( x_i - \overline{X} \right)^2}$

In [8]:
def std(X, ddof=0):
    '''standart variable'''
    
    MX = sum(X)/len(X)
    Sum = sum([(x - MX)**2 for x in X])
    
    return (Sum/(len(X)-ddof))**(1/2)

In [9]:
def corr(X, Y):
    '''Pearson correlation coefficient'''
    
    return cov(X, Y)/(std(X)*std(Y))

In [10]:
pearson = corr(X=salary, Y=scoring)
pearson

0.8874900920739162

In [11]:
df = pd.DataFrame({'salary':salary, 'scoring':scoring})
df.corr() # check

,salary,scoring
salary,1.00000,0.88749
scoring,0.88749,1.00000


### Задача 2

#### Проведите тест на значимость коэффициента корреляции Пирсона, найденного в предыдущей задаче. Что для этого нужно знать:

- Нулевая гипотеза: реальный коэффициент корреляции равен 0. Альтернативная гипотеза двухсторонняя.
- Статистика: t = r * sqrt(n - 2) / sqrt(1 - r ** 2), где r - коэффициент корреляции Пирсона, посчитанный по выборке.
- В предположении верности нулевой гипотезы эта статистика имеет распределение Стьюдента с параметром df = n - 2.

$
H_0 = 0\\
H_1 < 0 < H_1\\
t = \frac{r \sqrt{n - 2}}{\sqrt{1 - r^2}}
$

In [12]:
def statistic(r, n):
    
    return r * (n-2)**(1/2) / (1-r**2)**(1/2)

In [13]:
n = len(salary)

t = statistic(r=pearson, n=n)
t

5.447168150485575

In [14]:
alpha = [0.1, 0.05, 0.01][2]

t1 = stats.t.ppf(alpha / 2, df=n - 2)
t2 = stats.t.ppf(1 - alpha / 2, df=n - 2)

t1, t2

(-3.3553873313333966, 3.3553873313333957)

In [15]:
t1<=t<=t2

False

### Задача 3

#### Измерены значения IQ выборки студентов, обучающихся в местных технических вузах:
    131, 125, 115, 122, 131, 115, 107, 99, 125, 111
#### Известно, что в генеральной совокупности IQ распределен нормально. Найдите доверительный интервал для математического ожидания с надежностью 0.95.

$t = \dfrac{\overline{X} - \mu}{\sigma_X / \sqrt{n}},$

$
P \left( \overline{X} + t_{a/2} \cdot \cfrac{\sigma_X}{\sqrt{n}} \leq \mu \leq \overline{X} + t_{1-a/2} \cdot \cfrac{\sigma_X}{\sqrt{n}} \right) = p
$

In [16]:
iq = [131, 125, 115, 122, 131, 115, 107, 99, 125, 111]
p = 0.95 # надёжность

In [17]:
def confidence_interval(array, p):
    '''looking for confidence interval'''
    
    n = len(array) # объём выборки
    X = sum(array)/n # выборочное среднее
    sigma = std(X=array, ddof=1) # неизвестная сигма (несмещённое среднеквадратическое отклонение)
    df = n-1
    alpha = 1-p
    
    t1 = stats.t.ppf(alpha/2, df=df)
    t2 = stats.t.ppf(1-alpha/2, df=df)
    
    return X + t1*(sigma/n**(1/2)), X + t2*(sigma/n**(1/2))

In [18]:
confidence_interval(array=iq, p=p)

(110.55608365158724, 125.64391634841274)

### Задача 4

#### Выберите тему для проектной работы по курсу Теории вероятностей и математической статистики и напишите ее в комментарии к Практическому заданию.